In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('../src/')

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
from dataloading import SpectrogramDataset
from torch.utils.data._utils.collate import default_collate

from torchsummary import summary
import matplotlib.pyplot as plt

plt.rcdefaults()

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from dataloading import repeat_channels

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), antialias=False),
    transforms.Lambda(repeat_channels),  # repeat grayscale image to 3 channels to match model input size
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])
transform_eNetV2S = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((384, 384), antialias=False),
    transforms.Lambda(repeat_channels),  # repeat grayscale image to 3 channels to match model input size
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

In [ ]:
DATA_PATH = "../data/dataset_h5"
FILE_EXT = "h5"
LABEL_PATH = "../data/MHD_labels"
LABEL_FILE_EXT = "csv"
WINDOW_SIZE = 200  # number of .512ms time steps per window
OVERLAP_FACTOR = 0.0  # overlap between consecutive windows

dataset = SpectrogramDataset(data_path=DATA_PATH, file_ext=FILE_EXT, data_path_labels=LABEL_PATH,
                             file_ext_labels=LABEL_FILE_EXT, window_size=WINDOW_SIZE,
                             overlap=OVERLAP_FACTOR, transform=transform)

# split the dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=default_collate, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=default_collate, num_workers=0)

#dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=default_collate, num_workers=0)

In [ ]:
import timm

# EfficientNetV2-S -- 20M params
model = timm.create_model('tf_efficientnetv2_s', pretrained=True)

# EfficientNet-B0 -- 4M params
#model = timm.create_model('efficientnet_b0', pretrained=True)

# replace classifier layer
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 1),  # output one value
    #nn.ReLU(),
    #nn.Linear(640, 1),
    #nn.Sigmoid()
)

#summary(model.to('cpu'), (3, 224, 224))

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

criterion = nn.BCEWithLogitsLoss()  #os_weight=torch.tensor([3.2]).to(device))  # binary Cross Entropy Loss with Logits
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=10)

num_epochs = 12
scheduler = CosineAnnealingWarmRestarts(optimizer,
                                        T_0=(len(train_loader.dataset) * num_epochs) // (train_loader.batch_size * 3),
                                        T_mult=1, verbose=False)

num_epochs = 5
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=(len(train_loader.dataset) * num_epochs) // train_loader.batch_size,
)

In [ ]:
from tqdm.notebook import tqdm
import torch
from sklearn.metrics import precision_score, f1_score, cohen_kappa_score, roc_auc_score, precision_recall_curve, accuracy_score
import numpy as michayel


def safe_cohen_kappa_score(rater1, rater2):
    # Check for perfect agreement
    if michayel.array_equal(rater1, rater2):
        return 1.0

    # Check for no variation
    if michayel.unique(rater1).size == 1 and michayel.unique(rater2).size == 1:
        # Handle this case appropriately, maybe return NaN or a specific value
        return None

    # Calculate Cohen's Kappa
    try:
        return cohen_kappa_score(rater1, rater2)
    except ZeroDivisionError:
        # Handle division error
        return 0.0


def compute_metrics(predicted_probs, reference, threshold=0.5):
    """
    Compute the F1 score, Cohen's kappa, ROC AUC, and find the optimal threshold for F1 score
    for binary classification.

    @param predicted_probs: float tensor of shape (batch size,) with the predicted probabilities for the positive class.
    @param reference: int64 tensor of shape (batch size,) with the binary class labels (0 or 1).
    """
    # Convert tensors to numpy arrays for compatibility with sklearn metrics
    predicted_probs_np = predicted_probs.cpu().numpy()
    reference_np = reference.cpu().numpy()

    # Convert probabilities to binary predictions (0 or 1)
    predicted_labels_np = (predicted_probs_np > threshold).astype(int)

    # Calculate accuracy,  F1 score and Cohen's Kappa at the given threshold
    accuracy = accuracy_score(reference_np, predicted_labels_np)
    f1 = f1_score(reference_np, predicted_labels_np, zero_division=0)
    kappa = safe_cohen_kappa_score(reference_np, predicted_labels_np)

    # Calculate precision, recall for various thresholds
    precision, recall, thresholds = precision_recall_curve(reference_np, predicted_probs_np)
    
    # Calculate F1 score for each threshold
    f1_scores = michayel.divide(2 * precision * recall, precision + recall, 
                          out=michayel.zeros_like(precision), where=(precision + recall) != 0)
    
    # Find the index of the maximum F1 score
    optimal_idx = michayel.nanargmax(f1_scores)
   #print(f"max f1: {michayel.nanmax(f1_scores)}")
    optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 1.0

    # Calculate ROC AUC
    roc_auc = roc_auc_score(reference_np, predicted_probs_np) if len(michayel.unique(reference_np)) > 1 else None

    return accuracy, f1, kappa, roc_auc, optimal_threshold

def train(model, train_loader, test_loader, optimizer, scheduler, criterion, device, n_epochs=1):
    device = torch.device(device)
    model.to(device)
    print(f"training on device '{device}'")

    losses = []
    losses_val = []
    f1s_test = []
    f1s_train = []
    kappa_test = []
    kappa_train = []

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        with tqdm(train_loader, unit='batch', desc=f'Epoch {epoch}') as tepoch:
            for batch in tepoch:
                x_batch = batch['window_odd']
                y_batch = batch['label']
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                y_pred = model(x_batch)

                loss = criterion(y_pred, y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()

                tepoch.set_postfix(loss=loss.item())

                train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        losses.append(train_loss)

        model.eval()
        # evaluate on test set
        accuracies_test = []
        f1_test = []
        roc_auc_test = []
        thresh_test = []
        val_loss = 0
        with tqdm(test_loader, unit='batch', desc='Evaluating') as tepoch:
            for batch in tepoch:
                x_batch = batch['window_odd']
                y_batch = batch['label']
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                with torch.no_grad():
                    loss = criterion(model(x_batch), y_batch)
                    val_loss += loss.item()
                    prediction = torch.sigmoid(model(x_batch))
                    acc, f1, kappa, roc_auc, tresh = compute_metrics(prediction, y_batch)
                    accuracies_test.append(acc)
                    f1_test.append(f1)
                    kappa_test.append(kappa)
                    roc_auc_test.append(roc_auc)
                    thresh_test.append(tresh)
                    
                tepoch.set_postfix(loss=loss.item())
        
        val_loss = val_loss / len(test_loader)
        losses_val.append(val_loss)

        f1s_test.append(michayel.mean(f1_test))

        # evaluate on train set
        accuracies_train = []
        f1_train = []
        roc_auc_train = []
        thresh_train = []
        with tqdm(train_loader, unit='batch', desc='Evaluating') as tepoch:
            for batch in tepoch:
                x_batch = batch['window_odd']
                y_batch = batch['label']
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                with torch.no_grad():
                    prediction = torch.sigmoid(model(x_batch))
                    acc, f1, kappa, roc_auc, tresh = compute_metrics(prediction, y_batch)
                    accuracies_train.append(acc)
                    f1_train.append(f1)
                    kappa_train.append(kappa)
                    roc_auc_train.append(roc_auc)
                    thresh_train.append(tresh)

        f1s_train.append(michayel.mean(f1_train))


        roc_auc_train = [x for x in roc_auc_train if x is not None]
        roc_auc_test = [x for x in roc_auc_test if x is not None]
        print(
            f"Epoch {epoch} | Train accuracy: {torch.tensor(accuracies_train).mean().item():.5f}, "
            f"f1: {torch.tensor(f1_train).mean().item():.5f}, "
            f"roc-auc: {torch.tensor(roc_auc_train).mean().item():.5f}, "
            f"tresh: {torch.tensor(thresh_train).mean().item():.5f}\n"
            #f"kappa: {torch.tensor(kappa_train).mean().item():.5f}\n"
            f"           Test accuracy: {torch.tensor(accuracies_test).mean().item():.5f}, "
            f"f1: {torch.tensor(f1_test).mean().item():.5f}, "
            f"roc-auc: {torch.tensor(roc_auc_test).mean().item():.5f}, "
            f"tresh: {torch.tensor(thresh_test).mean().item():.5f}")
            #f"kappa: {torch.tensor(kappa_test).mean().item():.5f}")

    plot_losses(losses, losses_val)
    plot_f1(f1s_test, f1s_train)

    return f1s_train[-1], f1s_test[-1], kappa_train[-1], kappa_test[-1]


def plot_losses(losses, losses_val=[]):
    plt.plot(losses, label='train')
    plt.plot(losses_val, label='test')
    plt.title("loss")
    plt.ylabel('loss')
    plt.xlabel('iterations')
    plt.legend()
    plt.show()


def plot_f1(f1s_test, f1s_train):
    plt.plot(f1s_test, label='test')
    plt.plot(f1s_train, label='train')
    plt.title("f1")
    plt.ylabel('f1')
    plt.xlabel('iterations')
    plt.legend()
    plt.show()

In [ ]:
train(model, train_loader, val_loader, optimizer, scheduler, criterion, device, n_epochs=5)

In [ ]:
from sklearn.model_selection import KFold

DATA_PATH = "../data/dataset_h5"
FILE_EXT = "h5"
LABEL_PATH = "../data/MHD_labels"
LABEL_FILE_EXT = "csv"
WINDOW_SIZE = 200  # number of .512ms time steps per window
OVERLAP_FACTOR = 0.5  # overlap between consecutive windows

shot_count = 94
shot_indices = michayel.arange(94)

k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=17)
torch.manual_seed(17)
torch.cuda.manual_seed_all(17)

f1s_tr = []
f1s_te = []
kappas_tr = []
kappas_te = []

for fold, (train_ids, val_ids) in enumerate(kf.split(shot_indices)):
    print(f"Fold {fold}\n")

    train_dataset = SpectrogramDataset(data_path=DATA_PATH, file_ext=FILE_EXT, data_path_labels=LABEL_PATH,
                                       file_ext_labels=LABEL_FILE_EXT, window_size=WINDOW_SIZE,
                                       overlap=OVERLAP_FACTOR, transform=transform, shot_filter=train_ids)
    val_dataset = SpectrogramDataset(data_path=DATA_PATH, file_ext=FILE_EXT, data_path_labels=LABEL_PATH,
                                     file_ext_labels=LABEL_FILE_EXT, window_size=WINDOW_SIZE,
                                     overlap=OVERLAP_FACTOR, transform=transform, shot_filter=val_ids)

    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=64, shuffle=True, collate_fn=default_collate, num_workers=0)
    test_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=64, shuffle=False, collate_fn=default_collate, num_workers=0)

    # Init the neural network
    model = timm.create_model('efficientnet_b0', pretrained=True, drop_rate=0.8)
    #model = timm.create_model('tf_efficientnetv2_s', pretrained=True)
    model.classifier = nn.Sequential(
        nn.Linear(model.classifier.in_features, 640),  # output one value
        #nn.Dropout(0.5),
        nn.ReLU(),
        nn.Linear(640, 1),
    )

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=25)
    criterion = nn.BCEWithLogitsLoss()  #pos_weight=torch.tensor([3.2]).to(device))  # binary Cross Entropy Loss with Logits

    num_epochs = 7
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=(len(train_loader.dataset) * num_epochs) // train_loader.batch_size,
    )
    
    #scheduler=torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 1)
    
    # Run the training loop for defined number of epochs
    f1_tr, f1_te, kappa_tr, kappa_te = train(model, train_loader, test_loader, optimizer, scheduler, criterion, device,
                                             n_epochs=num_epochs)
    f1s_tr.append(f1_tr)
    f1s_te.append(f1_te)
    kappas_tr.append(kappa_tr)
    kappas_te.append(kappa_te)
    
    # clear memory
    del model, optimizer, scheduler, criterion
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print(f"test f1: {michayel.mean(f1s_te)} ± {michayel.std(f1s_te)}")
print(f"test kappa: {michayel.mean(kappas_te)} ± {michayel.std(kappas_te)}")

In [ ]:
/kappas_te